# Sortable Collections
[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.org/github/matyama/pfds/blob/main/notebooks/sortable.ipynb)

|       instance       | persistence | amortization | empty |     add    |  sort |
|:--------------------:|:-----------:|:------------:|:-----:|:----------:|:-----:|
|      Merge Sort      |  persistent |      yes     |  O(1) | O(log(n))* | O(n)* |
| Scheduled Merge Sort |  persistent |      no      |  O(1) |  O(log(n)) |  O(n) |

In [ ]:
class Sortable s where
    
    -- | Construct new (empty) collection
    empty :: Ord a => s a
    
    -- | Add new item to an existing sorted collection
    add :: Ord a => a -> s a -> s a
    
    -- | Convert a (sorted) collection into a sorted list
    sort :: Ord a => s a -> [a]

## Bottom-up Merge Sort
An instance of `Sortable` that is based on a [bottom-up *Merge Sort*](https://en.wikipedia.org/wiki/Merge_sort#Bottom-up_implementation).

In [ ]:
data MergeSort a = MS Int [[a]]

-- | Merge two sorted lists together.
mrg :: Ord a => [a] -> [a] -> [a]
mrg [] ys = ys
mrg xs [] = xs
mrg xs @ (x:xs') ys @ (y:ys') =
    if x <= y then x : mrg xs' ys
    else y : mrg xs ys'

instance Sortable MergeSort where

    -- | Construct new (empty) collection in O(1).
    empty = MS 0 []
    
    -- | Add new item to an existing sorted collection in O(log(n)) amortized time.
    add x (MS size segs) = MS (size + 1) (addSeg [x] segs size)
        where
            addSeg seg segs size =
                if even size then seg : segs
                else addSeg (mrg seg (Prelude.head segs)) (Prelude.tail segs) (size `div` 2)
    
    -- | Convert a (sorted) collection into a sorted list in O(n) amortized steps.
    sort (MS size segs) = foldl mrg [] segs

## Scheduled Bottom-up Merge Sort
An instance of `Sortable` that improves on `MergeSort` by incorporating *scheduling* to make the bounds worst case rather than amortized.

In [ ]:
-- | Schedule = [unevaluated suspensions per merge call]
type Schedule a = [[a]]

-- | SMS = (size, [(segment, segment's schedule)])
data ScheduledMergeSort a = SMS Int [([a], Schedule a)]

-- | Execute one merge step from a schedule by matching againt the head of the front stream.
-- | 
-- | Note: Since only the first list in a schedule can ever be empty the second clause, 
-- |       which executes next stream from the schedule if one ends, this function never loops.
exec1 :: Schedule a -> Schedule a
exec1 [] = []
exec1 ([]:schedule) = exec1 schedule
exec1 ((_:xs):schedule) = xs : schedule

-- | Execute the schedule twice and returns resulting schdule paired with the input segment.
exec2 :: [a] -> Schedule a -> ([a], Schedule a)
exec2 xs schedule = (xs, exec1 $ exec1 schedule)

instance Sortable ScheduledMergeSort where

    -- | Construct new (empty) collection in O(1).
    empty = SMS 0 []
    
    -- | Add new item to an existing sorted collection in O(log(n)) worst case time.
    add x (SMS size segs) = SMS (size + 1) (map (uncurry exec2) segs')
        where
            addSeg xs segs size rsched =
                if even size then (xs, reverse rsched) : segs
                else let ((xs', []):segs') = segs
                         xs'' = mrg xs xs'
                     in addSeg xs'' segs' (size `div` 2) (xs'':rsched)
            segs' = addSeg [x] segs size []
    
    -- | Convert a (sorted) collection into a sorted list in O(n) worst case steps.
    sort (SMS size segs) = mrgAll [] segs
        where
            mrgAll xs [] = xs
            mrgAll xs ((xs', _):segs) = mrgAll (mrg xs xs') segs